In [121]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np

In [122]:
df = pd.read_csv(r"E:\New folder (2)\DATAsets\titanic_dataset.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
40,41,0,3,"Ahlin, Mrs. Johan (Johanna Persdotter Larsson)",female,40.0,1,0,7546,9.4750,NaN,S
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S
823,824,1,3,"Moor, Mrs. (Beila)",female,27.0,0,1,392096,12.4750,E121,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q
852,853,0,3,"Boulos, Miss. Nourelain",female,9.0,1,1,2678,15.2458,NaN,C


In [123]:
df = df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

In [135]:
from sklearn.model_selection import train_test_split
x_test,x_train,y_test,y_train = train_test_split(df.drop(columns=["Survived"]),df["Survived"],random_state=42,train_size=0.20)
x_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
761,3,male,41.0,0,0,7.1250,S
645,1,male,48.0,1,0,76.7292,C
754,2,female,48.0,1,2,65.0000,S
556,1,female,48.0,1,0,39.6000,C
850,3,male,4.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [136]:
#filling the missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    int64   
 1   Pclass    891 non-null    int64   
 2   Sex       891 non-null    category
 3   Age       714 non-null    float64 
 4   SibSp     891 non-null    int64   
 5   Parch     891 non-null    int64   
 6   Fare      891 non-null    float64 
 7   Embarked  889 non-null    object  
dtypes: category(1), float64(2), int64(4), object(1)
memory usage: 49.8+ KB


In [137]:
df["Sex"] = df["Sex"].astype("category") #memory optimisation
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,NaN,1,1,15.2458,C
439,2,male,31.00,0,0,10.5000,S
840,3,male,20.00,0,0,7.9250,S
720,2,female,6.00,0,1,33.0000,S
39,3,female,14.00,1,0,11.2417,C
...,...,...,...,...,...,...,...
639,3,male,NaN,1,0,16.1000,S
878,3,male,NaN,0,0,7.8958,S
824,3,male,2.00,4,1,39.6875,S
803,3,male,0.42,0,1,8.5167,C


In [138]:
si_age = SimpleImputer() # filling mean value of age
si_embarked = SimpleImputer(strategy="most_frequent") # filling mode of embarked value

In [139]:
x_train_age = si_age.fit_transform(x_train[["Age"]])
X_train_embarked = si_embarked.fit_transform(x_train[["Embarked"]]) #i dont know the reason why double [[]] is used

In [140]:
x_test_age = si_age.transform(x_test[["Age"]])
x_test_embarked= si_embarked.transform(x_test[["Embarked"]])

In [141]:
x_train_age = x_train_age.astype("int")
x_test_age = x_test_age.astype("int")
x_train.drop(columns=["Age","Embarked"],inplace=True)
x_test.drop(columns=["Age","Embarked"],inplace=True)

In [142]:
x_train = pd.DataFrame(np.concatenate((np.array(x_train),np.array(x_train_age),np.array(x_train_embarked)),axis=1))
x_train

,0,1,2,3,4,5,6
0,3,male,1,1,15.2458,29,C
1,2,male,0,0,10.5,31,S
2,3,male,0,0,7.925,20,S
3,2,female,0,1,33.0,6,S
4,3,female,1,0,11.2417,14,C
...,...,...,...,...,...,...,...
708,3,male,1,0,16.1,29,S
709,3,male,0,0,7.8958,29,S
710,3,male,4,1,39.6875,2,S
711,3,male,0,1,8.5167,0,C


In [143]:
x_test = pd.DataFrame(np.concatenate((np.array(x_test),np.array(x_test_age),np.array(x_test_embarked)),axis=1))
x_test

,0,1,2,3,4,5,6
0,3,male,0,0,7.125,41,S
1,1,male,1,0,76.7292,48,C
2,2,female,1,2,65.0,48,S
3,1,female,1,0,39.6,48,C
4,3,male,4,2,31.275,4,S
...,...,...,...,...,...,...,...
173,3,female,0,0,7.65,21,S
174,1,male,0,0,31.0,29,S
175,3,male,2,0,14.1083,41,S
176,1,female,1,2,120.0,14,S


In [14]:
#applying one hot encoding
ohe_sex = OneHotEncoder(handle_unknown="ignore",sparse_output=False)
ohe_embarked = OneHotEncoder(handle_unknown="ignore",sparse_output=False)


In [16]:
male_female_train = ohe_sex.fit_transform(x_train[["Sex"]])
male_female_test = ohe_sex.fit(x_test[["Sex"]])


In [17]:
Embarked_train = ohe_embarked.fit_transform(x_train[["Embarked"]])
Embarked_test = ohe_embarked.fit(x_test[["Embarked"]])

In [18]:
#in decision tree we dont have to remove one columns as there is no case of multiple collinearity
x_train.drop(columns=["Sex","Embarked"],inplace=True)
x_test.drop(columns=["Sex","Embarked"],inplace=True)

In [25]:
df.iloc[711]

Survived        0
Pclass          1
Sex          male
Age           NaN
SibSp           0
Parch           0
Fare        26.55
Embarked        S
Name: 711, dtype: object

In [19]:
x_train_transformed = np.concatenate((x_train,male_female_train,Embarked_train),axis=1)
x_test_transformed = np.concatenate((x_test,male_female_test,Embarked_test),axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [20]:
pd.DataFrame(x_train_transformed)

,0,1,2,3,4,5,6,7,8,9,10
0,3.0,1.0,1.0,15.2458,29.531933,0.0,1.0,0.0,0.0,1.0,0.0
1,2.0,0.0,0.0,10.5000,31.000000,0.0,1.0,0.0,0.0,1.0,0.0
2,3.0,0.0,0.0,7.9250,20.000000,0.0,1.0,0.0,0.0,0.0,1.0
3,2.0,0.0,1.0,33.0000,6.000000,1.0,0.0,0.0,0.0,0.0,1.0
4,3.0,1.0,0.0,11.2417,14.000000,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
708,3.0,1.0,0.0,16.1000,29.531933,0.0,1.0,0.0,0.0,1.0,0.0
709,3.0,0.0,0.0,7.8958,29.531933,0.0,1.0,0.0,0.0,0.0,1.0
710,3.0,4.0,1.0,39.6875,2.000000,0.0,1.0,0.0,0.0,0.0,1.0
711,3.0,0.0,1.0,8.5167,0.420000,0.0,1.0,0.0,0.0,0.0,1.0


In [154]:
#Model_bulding
dt = DecisionTreeClassifier()

In [155]:
dt.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [156]:
y_predict=dt.predict(x_test_transformed)

In [157]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.6910112359550562